# RAG with Azure AI Search

<img src="./architecture.png">

<img src="vector_embeddings.svg" >


### Installing Packages

In [ ]:
%pip install openai

### Loading variables from the .env file

In [ ]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_API_KEY")

### Creating an Azure OpenAI Client

In [ ]:
from openai import AzureOpenAI

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")  
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")

azure_openai_client = AzureOpenAI(
    api_key=azure_openai_key,
    api_version="2024-02-15-preview",
    azure_endpoint=azure_openai_endpoint
)

### Creating Embedding Generator Function

In [ ]:
def generate_embeddings(client, text):
    embedding_model = os.getenv("EMBEDDING_ENGINE")
    
    response = client.embeddings.create(
        input=text,
        model = embedding_model
    )
    
    embeddings=response.model_dump()
    return embeddings['data'][0]['embedding']
    

In [ ]:
user_query = "What is the review of the creek hotel in Dubai?"
vectorised_user_query = generate_embeddings(azure_openai_client, user_query)
print(vectorised_user_query)


In [ ]:
context=[]

### Sending API call to the Search Index

In [ ]:
import requests
import json


url = f"{service_endpoint}/indexes/{index_name}/docs/search?api-version=2023-11-01"
    
headers = {
        "Content-Type": "application/json",
        "api-key": key
    }
    
body =   {
        "count": True,
        "select": "chunk",
        "vectorQueries": [
            {
                "vector": vectorised_user_query,
                "k": 3,
                "fields": "text_vector",
                "kind": "vector"
            }
        ]
    }
    
response = requests.post(url, headers=headers, data=json.dumps(body))
documents = response.json()['value']

for doc in documents:
    context.append(dict(
        {
            "chunk": doc['chunk'],
            "score": doc['@search.score']
            
        }
    ))
    
for doc in context:
    print(doc)



### Calling GPT Engine for Summarisation

In [ ]:
system_prompt = f""""You are meant to behave as a RAG chatbot that derives its context from a database of hotel reviews stored in Azure AI Search Solution.
please answer strictly from the context from the database provided and if you dont have an answer please politely say so. dont include any extra 
information that is not in the context and dont include links as well.
the context passed to you will be in the form of a pythonic list with each object in the list containing details of hotel reviews and
having structure as follows:

 "chunk": "the content of the review",
 "score": "the relevancy score of the review"


the pythonic list contains best 3 matches to the user query based on cosine similarity of the embeddings of the user query and the review descriptions.
please structure your answers in a very professional manner and in such a way that the user does not get to know that its RAG working under the hood
and its as if they are talking to a human. """

user_prompt = f""" the user query is: {user_query}
the context is : {context}"""

chat_completions_response = azure_openai_client.chat.completions.create(
    model = os.getenv("GPT_ENGINE"),
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.7
)

print(chat_completions_response.choices[0].message.content)

